# Setup

In [ ]:
!pip install scikit-optimize
import pickle
import pandas as pd
import numpy as np
from tqdm import tqdm
import torch
import torch.nn as nn
from sklearn.metrics import brier_score_loss
from sklearn.model_selection import KFold
import skopt
import matplotlib.pyplot as plt
print(torch.cuda.is_available())

from google.colab import drive
drive.mount('/content/drive')

True
Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
# load data
with open('/content/drive/MyDrive/Colab_Projects/AML_final_project/data/data_train.pt', 'rb') as file:
    X, y = pickle.load(file)

data = np.concatenate((X,y), axis=1)
data = torch.tensor(data).float()
data = data.to("cuda")

# Create Network Model

In [ ]:
class MLP(nn.Module):
  '''implements a multilayer perceptron with sigmoid output activation
     input_dim    --> dimension of input vector (number of dataframe columns)
     hidden_dim   --> dimension of hidden layers
     hidden_depth --> number of hidden layers '''

  def __init__(self, input_dim, hidden_dim, hidden_depth):
    super(MLP, self).__init__()

    self.input_dim    = input_dim
    self.hidden_dim   = hidden_dim
    self.hidden_depth = hidden_depth

    self.layers = nn.ModuleDict()

    self.layers['fcl_in'] = nn.Linear(self.input_dim, self.hidden_dim) 

    for i in range(self.hidden_depth):
      self.layers['fcl_hidden_' + str(i)] = nn.Linear(self.hidden_dim, self.hidden_dim)

    self.layers['fcl_out']  = nn.Linear(self.hidden_dim, 2)


  def forward(self, x):
    x = nn.functional.relu(self.layers['fcl_in'](x))

    for i in range(self.hidden_depth):
      x = nn.functional.relu(self.layers['fcl_hidden_' + str(i)](x))

    x = torch.sigmoid(self.layers['fcl_out'](x))

    return x

# Define Training Function

In [ ]:
INPUT_DIM = X.shape[1]
N_EPOCHS  = 40
LOSS      = nn.BCELoss()
N_SPLITS  = 5

def train(hyper_list):
  LEARNING_RATE = hyper_list[0]
  BATCH_SIZE = int(hyper_list[1])
  HIDDEN_DIM = int(hyper_list[2])
  HIDDEN_DEPTH = int(hyper_list[3])

  # initializing network and optimizer
  net = MLP(INPUT_DIM, HIDDEN_DIM, HIDDEN_DEPTH).to("cuda")
  optim = torch.optim.Adam(net.parameters(), lr = LEARNING_RATE)

  # cross validaton splitting
  kf = KFold(n_splits=N_SPLITS, shuffle=True, random_state=20210927)
  
  # cross validation loop
  cross_score = np.empty((N_SPLITS,))
  for i, (train_index, test_index) in enumerate(kf.split(X)):
    data_train = data[train_index]
    data_test  = data[test_index]
    train_generator = torch.utils.data.DataLoader(dataset = data_train,
                                                  batch_size = BATCH_SIZE,
                                                  shuffle = True)
    # epoch loop
    for _ in range(N_EPOCHS):

      # batch loop
      for batch in train_generator:
        optim.zero_grad()
        target     = batch[:,[0,1]]
        prediction = net(batch[:, 2:]) 
        loss = LOSS(prediction, target)
        loss.backward()
        optim.step()

    # calculate score for this cross validation instance
    with torch.no_grad():
      target_test      = data_test[:,[0,1]]
      prediction_test  = net(data_test[:, 2:])
      cross_score[i] = LOSS(prediction_test, target_test)
  
  return np.mean(cross_score)

# Run GP Hyperparameter Search

In [ ]:
optim_res = skopt.gp_minimize(func=train,
                              dimensions=[(0.0001, 0.005), (100,200), (8,128), (1,5)],
                              n_calls=15,
                              n_initial_points=5,
                              acq_func="EI",
                              verbose=True
                              )

Iteration No: 1 started. Evaluating function at random point.
Iteration No: 1 ended. Evaluation done at random point.
Time taken: 6477.4010
Function value obtained: 32.9502
Current minimum: 32.9502
Iteration No: 2 started. Evaluating function at random point.


In [ ]:
skopt.dump(optim_res, '/content/drive/MyDrive/Colab_Projects/AML_final_project/data/gp_result.pkl')